# Importações 

In [30]:
#pip install pandas matplotlib scikit-learn

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer

In [32]:
df =  pd.read_csv("movies.csv")

# Filtros

## Filmes ja lançados (status = Released)
Filmes que ainda não foram lançados não possuem dados importantes para a análise

In [33]:
registros_antes = len(df)
df = df[df['status'] == 'Released']
registros_depois = len(df)
print(f"Registros perdidos com o filtro (status): {registros_antes - registros_depois}")

Registros perdidos com o filtro (status): 5223


## Registros duplicados
Não faz sentido ter o mesmo filme mais de uma vez

In [34]:
registros_antes = len(df)
df = df.drop_duplicates()
registros_depois = len(df)
print(f"Registros perdidos com o filtro (duplicados gerais): {registros_antes - registros_depois}")

Registros perdidos com o filtro (duplicados gerais): 0


### Duplicados por titulo e data de lançamento

In [35]:
registros_antes = len(df)
df = df.drop_duplicates(subset=['title', 'release_date'])
registros_depois = len(df)
print(f"Registros perdidos com o filtro (titulo + data): {registros_antes - registros_depois}")

Registros perdidos com o filtro (titulo + data): 61422


In [36]:
exemplo = df[df['title'].isin(['The Eighth Clause', 'Battle: Freestyle'])].sort_values(by='title')
exemplo.head()

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
126,785521,Battle: Freestyle,Romance-Drama,no,Amalie and Mikael lead their street dance team...,333.723,Friland Produksjon AS,2022-04-01,0.0,0.0,88.0,Released,NaN,5.1,28.0,Lisa Teige-Fabian Svegaard Tapia-Ellen Dorrit ...,paris france-based on novel or book-norway-dan...,/6D6QumiHEhnpZG12Ibjy2BxA6n4.jpg,/8WpRRiz3qQqzBiifCTm1dW0b6bs.jpg,818750-790525-800407-739993-946726-682344-9532...
13,956101,The Eighth Clause,Thriller,la,Kat and Borja appear to be a perfect couple bu...,2259.303,SDB Films-El Hombre Orquesta,2022-04-29,0.0,0.0,0.0,Released,NaN,4.6,10.0,Maite Perroni-Manuel Vega-Óscar Jaenada-Jessic...,NaN,/8tc8eMFAX2SDC1TRu987qFQy8Cl.jpg,/kLnqNE9Af5QHyvUxw8cDGhF1ilv.jpg,NaN


## Budget > 0
Filmes com budget igual a 0 ou negativo, representam inconssitência com as informações, trazendo erros para a analise

In [37]:
registros_antes = len(df)
df = df[df['budget'] > 0]
registros_depois = len(df)
print(f"Registros perdidos com o filtro (budget > 0): {registros_antes - registros_depois}")

Registros perdidos com o filtro (budget > 0): 622512


## Runtime > 70
Filmes com runtime menores que 70 não são considerados longa-metragem, que é nosso foco

In [38]:
registros_antes = len(df)
df = df[df['runtime'] > 70]
registros_depois = len(df)
print(f"Registros perdidos com o filtro (runtime > 70): {registros_antes - registros_depois}")

Registros perdidos com o filtro (runtime > 70): 12383


## Vote_count > 30
Filmes com poucos votos, podem levantar erros estatisticos sobre a analise dos filmes, pois o target 'vote_average' poderá ficar enviesado

In [39]:
registros_antes = len(df)
df = df[df['vote_count'] > 30]
registros_depois = len(df)
print(f"Registros perdidos com o filtro: {registros_antes - registros_depois}")

Registros perdidos com o filtro: 9956


## Removendo colunas que não serão utilizadas na análise
- Title -> não usaremos processamento textual
- Overview -> não usaremos processamento textual
- Production Companies -> Vale com o One Hot Encoding? 
- Revenue -> Vazamento de dados.
- Status -> registro unico agr, ent inutil
- Tagline -> não usaremos processamento textual e 85% dos registros são NaN
- vote_avarege -> TARGET 
- vote_count -> vazamento de dados.
- Credits -> Vale com o One Hot Encoding?
- Keywords -> não usaremos processamento textual e 71% dos registros são NaN
- Poster_path e Backdrop_path -> não faz sentido para a análise
- Recommendations -> não faz sentido para a análise, 95% dos valores são NaN, vazamento de dados

In [40]:
df = df.drop(columns=[
    'id','title', 'overview', 'revenue', 'status', 'tagline', 
    'vote_count', 'poster_path', 'backdrop_path', 'recommendations', 'keywords',
    'credits'
])

# Colunas que serão utilizadas na análise
print("Colunas que serão utilizadas na análise:")
print(df.columns)

Colunas que serão utilizadas na análise:
Index(['genres', 'original_language', 'popularity', 'production_companies',
       'release_date', 'budget', 'runtime', 'vote_average'],
      dtype='object')


## Limpando todos os valores nulos

In [41]:
registros_antes = len(df)
df = df.dropna()
registros_depois = len(df)
print(f"Registros perdidos com o filtro: {registros_antes - registros_depois}")

Registros perdidos com o filtro: 231


# To CSV
informações do dataset filtrado

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10590 entries, 0 to 598651
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   genres                10590 non-null  object 
 1   original_language     10590 non-null  object 
 2   popularity            10590 non-null  float64
 3   production_companies  10590 non-null  object 
 4   release_date          10590 non-null  object 
 5   budget                10590 non-null  float64
 6   runtime               10590 non-null  float64
 7   vote_average          10590 non-null  float64
dtypes: float64(4), object(4)
memory usage: 744.6+ KB


In [43]:
df.describe()

,popularity,budget,runtime,vote_average
count,10590.000000,1.059000e+04,10590.000000,10590.000000
mean,30.970914,2.392062e+07,108.099245,6.364052
std,181.275201,3.720596e+07,22.109043,0.905434
min,0.600000,1.000000e+00,71.000000,1.800000
25%,8.322250,3.000000e+06,93.000000,5.800000
50%,12.904500,1.000000e+07,103.000000,6.400000
75%,22.545000,2.800000e+07,118.000000,7.000000
max,8763.998000,4.600000e+08,608.000000,9.000000


In [ ]:
#df.to_csv('filmes_luan.csv', index=False)